In [1]:
import os
from dotenv import load_dotenv, find_dotenv
from langchain_community.graphs import Neo4jGraph
import openai

def setup():
    _ = load_dotenv(find_dotenv())
    try:
        openai.api_key = os.getenv("OPENAI_API_KEY")
    except:
        print("OPENAI_API_KEY not found,")

In [2]:
setup()

In [3]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph()

In [4]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Intermediary {id: STRING, name: STRING, country: STRING}
Officer {id: STRING, name: STRING, country: STRING}
Entity {id: STRING, name: STRING, jurisdiction: STRING, incorporation_date: DATE, status: STRING, country: STRING, source: STRING}
Relationship properties:
RELATIONSHIP {type: STRING}
The relationships:
(:Intermediary)-[:RELATIONSHIP]->(:Entity)
(:Officer)-[:RELATIONSHIP]->(:Entity)


In [5]:
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)

In [10]:
response = chain.invoke({"query": "Where are most of the entities located?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Entity)
RETURN e.country, COUNT(e) AS numEntities
ORDER BY numEntities DESC
LIMIT 1;
Full Context:
[{'e.country': 'British Virgin Islands', 'numEntities': 986}]

> Finished chain.


{'query': 'Where are most of the entities located?',
 'result': 'Most of the entities are located in the British Virgin Islands.'}

In [11]:
response = chain.invoke({"query": "Who is the officer connected to the most entities?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (o:Officer)-[:RELATIONSHIP]->(e:Entity)
WITH o, COUNT(e) AS numEntities
RETURN o.name, numEntities
ORDER BY numEntities DESC
LIMIT 1;
Full Context:
[{'o.name': 'Justine Catherine Leacacos', 'numEntities': 7}]

> Finished chain.


{'query': 'Who is the officer connected to the most entities?',
 'result': 'Justine Catherine Leacacos is the officer connected to the most entities.'}

In [13]:
response = chain.invoke({"query": "How many intermediates deal with PACIFIC ASIA OVERSEAS LTD.?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (i:Intermediary)-[:RELATIONSHIP]->(e:Entity)
WHERE e.name = 'PACIFIC ASIA OVERSEAS LTD.'
RETURN COUNT(i)
Full Context:
[{'COUNT(i)': 2}]

> Finished chain.


{'query': 'How many intermediates deal with PACIFIC ASIA OVERSEAS LTD.?',
 'result': 'There are 2 intermediates that deal with PACIFIC ASIA OVERSEAS LTD.'}

In [6]:
response = chain.invoke({"query": "How are JADE SUCCESS LIMITED and Blue Water Properties Holdings Limited connected?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH path = shortestPath((i:Intermediary)-[:RELATIONSHIP*]-(e:Entity {name: "JADE SUCCESS LIMITED"}))
MATCH path2 = shortestPath((o:Officer)-[:RELATIONSHIP*]-(e2:Entity {name: "Blue Water Properties Holdings Limited"}))
RETURN path, path2
Full Context:
[{'path': [{'country': 'CHN', 'name': 'Fang Fang', 'id': '82366'}, 'RELATIONSHIP', {'country': 'British Virgin Islands', 'jurisdiction': 'British Virgin Islands', 'name': 'GOLD SEARCH LIMITED', 'id': '125401', 'source': 'Offshore Leaks', 'incorporation_date': neo4j.time.Date(1996, 8, 23), 'status': 'Struck / Defunct / Deregistered'}, 'RELATIONSHIP', {'country': 'THA;VGB;IDN;SGP', 'name': 'Portcullis TrustNet (BVI) Limited', 'id': '54662'}, 'RELATIONSHIP', {'country': 'British Virgin Islands', 'jurisdiction': 'British Virgin Islands', 'name': 'JADE SUCCESS LIMITED', 'id': '126329', 'source': 'Offshore Leaks', 'incorporation_date': neo4j.time.Date(1997, 3, 19), 'status': 'Stru

{'query': 'How are JADE SUCCESS LIMITED and Blue Water Properties Holdings Limited connected?',
 'result': 'JADE SUCCESS LIMITED and Blue Water Properties Holdings Limited are connected through Portcullis TrustNet (BVI) Limited.'}